# 1. Remove dodgy images

In [ ]:
import cv2
import imghdr#拡張子をファイルの文字を参照するのではなく、構造を参照するファイル判定モジュール。
import tensorflow as tf

In [ ]:
#ディレクトリ名を指定
data_dir = 'snake_images' 

In [ ]:
#受けつける拡張子の指定
image_exts = ['jpeg','jpg', 'bmp', 'png']

In [ ]:
#.DS_Storeがあるか確認。
for image_class in os.listdir(data_dir): 
    print(image_class)

以下のコードで.DS_Storeディレクトリを消去する

In [ ]:
import os

for image_class in os.listdir(data_dir):
    if image_class == '.DS_Store':
        file_path = os.path.join(data_dir, image_class)
        os.remove(file_path)
    else:
        print(image_class)

In [ ]:
#.DS_Storeが消えているのを確認
for image_class in os.listdir(data_dir): 
    print(image_class)

In [ ]:
#対象としていない拡張子と壊れている画像ファイルをremoveする。
for image_class in os.listdir(data_dir): 
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try: 
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts: 
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e: 
            print('Issue with image {}'.format(image_path))
            os.remove(image_path)

0と1がどのクラスに対応しているのかを出力する。

In [ ]:
class_mapping = {}  # クラス名とラベルの対応付けを格納する辞書

for label, image_class in enumerate(os.listdir(data_dir)):
    if image_class == '.DS_Store':
        file_path = os.path.join(data_dir, image_class)
        os.remove(file_path)
    else:
        class_name = image_class.split('_')[0]
        print(f"{class_name}: {label}")
        class_mapping[label] = class_name

# class_mapping辞書を修正して、ラベルを0と1に変更する。
class_mapping = {label: label - 1 for label in class_mapping}


# 2. Load Data

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
#指定されたディレクトリから画像データセットを作成
data = tf.keras.utils.image_dataset_from_directory('snake_images')

In [ ]:
#データセットの要素をnumpy配列として返すiteratorを作成。
data_iterator = data.as_numpy_iterator()

In [ ]:
#イテレータから次のバッチを取得する
batch = data_iterator.next()

#batch0:画像データ
#batch1: 0 or1（毒あり or 毒なしラベル）

In [ ]:
#バッチの上から４つ選び、プロットする。
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

# 3. Scale Data 

In [ ]:
#x:image
#y:target
#scale data to 0 through 1
data = data.map(lambda x,y: (x/255, y))

In [ ]:
#データセットの要素をnumpy配列として返すiteratorを作成。
#イテレータから次のバッチを取得する
batch = data.as_numpy_iterator().next()

In [ ]:
#スケーリングしたデータセットのバッチの上から４つ選び、プロットする。
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img)
    ax[idx].title.set_text(batch[1][idx])

# 4. Split Data to train,test,vallidating data

In [ ]:
#バッチ数をプリント
len(data)

In [ ]:

train_size = int(len(data)*.7)
val_size = int(len(data)*.2)
test_size = int(len(data)*.1)+1 #hold until the end

以下の3変数の合計がlen(data)分になるようにする。

In [ ]:
len(train_size )

In [ ]:
len(val_size)

In [ ]:
len(test_size)

In [ ]:
#take():how much we take from dataset
#skip():usually,skip take() area 

train = data.take(train_size)#train model
val = data.skip(train_size).take(val_size)#evaluate model
test = data.skip(train_size+val_size).take(test_size)

# 5. Build Deep Learning Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

TensorFlowで新しいシーケンシャルモデルを作成する。

In [ ]:
#空のシーケンシャルモデルを作成
#シーケンシャルモデル:ニューラルネットワークモデルを構築するための層の線形スタック

model = Sequential()

In [ ]:
#add()メソッドを使用してモデルにレイヤーを追加します。
#各レイヤーは、フォワードパス中にデータが通過する特定の操作や変換を表す。

#これらをシーケンシャルクラスにスタックする。


 #Conv2D's args:(number of filters,(size of the filter),stride,activation func-name,input shape)
# 最初の層にはinput_shapeの入力が必要
model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3))) 
#2x2フィルタから最大値を取る。
model.add(MaxPooling2D())

model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

#1次元にする
model.add(Flatten())

#全結合層
#256個の値が出力されるようにする
model.add(Dense(256, activation='relu'))
#単一の出をと0か1にマップする。
model.add(Dense(1, activation='sigmoid'))


In [ ]:
#モデルのコンパイルを行う。

#Adamオプティマイザーを使用
#tf.losses.BinaryCrossentropy()を指定して損失関数として二値交差エントロピーを使用する。
#評価関数としてaccuracyを使用する.

model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [ ]:
#モデルの構造を表示。
model.summary()

# 6. Train

In [ ]:
#ログファイルを保存するディレクトリを指定
logdir='logs'

In [ ]:
#fit間数にわたすため、コールバックを作成
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
#fit:training
#epocs:how long we 're actually going to go ahead and train for

hist = model.fit(train, epochs=5, validation_data=val, callbacks=[tensorboard_callback])

In [ ]:
#返されたオブジェクトのhistory属性を使ってアクセス。
hist.history

# 7. Plot Performance

In [ ]:
#lossとval_lossをプロット

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
#accuracyとval_accuracyをプロット

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

# 8. Evaluate

学習済みモデルの評価を行う

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [ ]:
pre = Precision()
re = Recall()
acc = BinaryAccuracy()

In [ ]:
#精度、再現率、および二値分類精度を計算し、
#真のラベルyと予測されたラベルyhatを比べる
for batch in test.as_numpy_iterator(): 
    X, y = batch
    yhat = model.predict(X)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)

In [ ]:
#それぞれの出力値は1に近いほどよい
print(pre.result(), re.result(), acc.result())

# 9. Test

In [ ]:
import cv2

In [ ]:
result=model.predict(test)

In [ ]:
#モデルが一度も見たことがない画像1枚を渡す
img = cv2.imread('p.png')
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
#画像の前処理を行う
resize = tf.image.resize(img, (256,256))
plt.imshow(resize.numpy().astype(int))
plt.show()

In [ ]:
#前処理をした画像をモデルに入力し、予測を行う。
yhat = model.predict(np.expand_dims(resize/255, 0))

In [ ]:
#予測値が 0.5 より大きい場合は、「予測されたクラスは非毒性」と表示される。
#それ以外の場合は、「予測されたクラスは有毒」と表示。
if yhat > 0.5: 
    print(f'Predicted class is non-poisonous')
else:
    print(f'Predicted class is poisonous')

# 10. Save the Model

学習モデルの保存を行う。

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
#modelsフォルダに学習済みモデルを保存
model.save(os.path.join('models','snake_poisonClassifier1.h5'))

In [ ]:
#保存したモデルをloadする
new_model = load_model(os.path.join('models','snake_poisonClassifier1.h5'))

In [ ]:
#モデルが一度も見たことがない画像1枚を渡し、予測を行う。
yhatnew = new_model.predict(np.expand_dims(resize/255, 0))

In [ ]:
#予測値が 0.5 より大きい場合は、「予測されたクラスは非毒性」と表示される。
#それ以外の場合は、「予測されたクラスは有毒」と表示。
if yhat > 0.5: 
    print(f'Predicted class is non-poisonous')
else:
    print(f'Predicted class is poisonous')